In [25]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import random
import zipfile
device='cpu'

In [26]:
def load_data_jay_lyrics():
    with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars=f.read().decode('utf-8')
    corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
    corpus_chars=corpus_chars[0:10000]
    idx_to_char=list(set(corpus_chars))
    char_to_idx=dict([( char,i ) for i , char in enumerate(idx_to_char)])
    vocab_size=len(char_to_idx)
    corpus_indices=[char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [27]:
corpus_indices,char_to_idx,idx_to_char,vocab_size=load_data_jay_lyrics()

In [28]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size

In [29]:
def get_params():
    def _one(shape):
        ts=torch.Tensor(np.random.normal(0,0.01,size=shape))
        return torch.nn.Parameter(ts,requires_grad=True)
    
    def _three():
        return (_one((num_inputs,num_hiddens)),
               _one((num_hiddens,num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens),requires_grad=True))
    
    W_xi,W_hi,b_i=_three()
    W_xf,W_hf,b_f=_three()
    W_xo,W_ho,b_o=_three()
    W_xc,W_hc,b_c=_three()
    
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,dtype=torch.float32),requires_grad=True)
    
    return nn.ParameterList([W_xi,W_hi,b_i,W_xf,W_hf,b_f,W_xo,W_ho,b_o,W_xc,W_hc,b_c,W_hq,b_q])

In [30]:
def init_lstm_state(batch_size,num_hiddens,device):
    return (torch.zeros(batch_size,num_hiddens),torch.zeros(batch_size,num_hiddens))

In [47]:
def lstm(inputs,state,params):
    [W_xi,W_hi,b_i,W_xf,W_hf,b_f,W_xo,W_ho,b_o,W_xc,W_hc,b_c,W_hq,b_q]=params
    (H,C)=state
    outputs=[]
    for X in inputs:
        I=torch.sigmoid(torch.matmul(X,W_xi)+torch.matmul(H,W_hi)+b_i)
        F=torch.sigmoid(torch.matmul(X,W_xf)+torch.matmul(H,W_hf)+b_f)
        O=torch.sigmoid(torch.matmul(X,W_xo)+torch.matmul(H,W_ho)+b_o)
        
        C_tilda=torch.tanh(torch.matmul(X,W_xc)+torch.matmul(H,W_hc)+b_c)
        C=F*C+I*C_tilda
        H=O*C.tanh()
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,C)
        

In [48]:
torch.Tensor([1,2,3,-1,-3,5]).tanh()

tensor([ 0.7616,  0.9640,  0.9951, -0.7616, -0.9951,  0.9999])

In [49]:
for i in [1,2,3,-1,-3,5,7]:
    print((np.exp(i)- np.exp(-i))/(np.exp(i)+np.exp(-i)))

0.7615941559557649
0.964027580075817
0.9950547536867306
-0.7615941559557649
-0.9950547536867306
0.999909204262595
0.9999983369439447


In [50]:
def to_onehot(x,n_class):
    def _one_hot(x,n_class):
        x=x.long()
        res=torch.zeros(x.shape[0],n_class,dtype=torch.float32)
        res.scatter_(1,x.view(-1,1),1)
        return res
    return [_one_hot(x[:,i],n_class) for i in range(x.shape[1])]

In [51]:
corpus_indices,char_to_idx,idx_to_char,vocab_size=load_data_jay_lyrics()

In [52]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size

In [53]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32,1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [54]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices=torch.Tensor(corpus_indices)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1)//num_steps
    for i in range(epoch_size):
        i=i*num_steps
        X=indices[:,i:i+num_steps]
        Y=indices[:,i+1:i+num_steps+1]
        yield X,Y
    

In [55]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx):
    state=init_rnn_state(1,num_hiddens,device)
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        x=to_onehot(torch.Tensor([[output[-1]]]),vocab_size)
        (y,state)=rnn(x,state,params)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [56]:
def grad_clipping(params,theta,device):
    norm=torch.Tensor([0.0])
    for p in params:
        norm +=(p.grad.data **2).sum()
    norm=norm.sqrt().item()
    if norm > theta:
        for p in params:
            p.grad.data *= (theta / norm)

In [57]:
def sgd(params,lr,batch_size):
    for p in params:
        p.data -= lr * p.grad /batch_size

In [58]:
for x,y in data_iter_consecutive(list(range(10)),2,3):
    print(x.shape)
    print(torch.stack(to_onehot(x,10)).shape)
    print(to_onehot(x,10))
#     print(x)
#     print(y)

torch.Size([2, 3])
torch.Size([3, 2, 10])
[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]), tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]), tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])]


In [59]:
torch.cat([a,b])

NameError: name 'a' is not defined

In [60]:
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,corpus_indices,idx_to_char,char_to_idx,is_random_iter,
                          num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,perfixes):
    if is_random_iter:
        data_iter_fn=data_iter_random
    else:
        data_iter_fn=data_iter_consecutive
    params=get_params()
    loss=nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:
            state=init_rnn_state(batch_size,num_hiddens,device)
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_fn(corpus_indices,batch_size,num_steps)
        for x,y in data_iter:
            if is_random_iter:
                state = init_rnn_state(batch_size,num_hiddens,device)
            else:
                for s in state :
                    s.detach_()
            inputs=to_onehot(x,vocab_size)
            (outputs,state)=rnn(inputs,state,params)
            outputs=torch.cat(outputs,dim=0)
            y=torch.transpose(y,0,1).contiguous().view(-1)
            l=loss(outputs,y.long())
            
            if params[0].grad is not None:
                for p in params:
                    p.grad.data.zero_()
            l.backward()
            grad_clipping(params,clipping_theta,device)
            sgd(params,lr,1)
            l_sum+=l.item()*y.shape[0]
            n+=y.shape[0]
            
        if (epoch +1) % pred_period ==0:
            print('l_sum : ',l_sum/n)
            print('epoch %d,perplexity %f,time %.2f sec ' % (epoch+1,math.exp(l_sum/n),time.time()-start))
            
            for perfix in perfixes:
                print('-',predict_rnn(perfix,pred_len,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx))
            
            
            

In [61]:
num_epochs,num_steps,batch_size,lr,clipping_theta=160,35,32,1e2,1e-2
pred_period,pred_len,prefixes=40,50,['分开','不分开']

In [62]:
train_and_predict_rnn(lstm,get_params,init_lstm_state,num_hiddens,vocab_size,
                     corpus_indices,idx_to_char,char_to_idx,None,
                     num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,
                     pred_len,prefixes)

l_sum :  5.35698127746582
epoch 40,perplexity 212.083757,time 3.24 sec 
- 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
- 不分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
l_sum :  4.154011607170105
epoch 80,perplexity 63.688984,time 2.43 sec 
- 分开 我想你你想你 我不要这我 我不要这我 我不要这我 我不要这我 我不要这我 我不要这我 我不要这我 
- 不分开 你知我的爱爱 你不好觉 我不要这我 我不要这我 我不要这我 我不要这我 我不要这我 我不要这我 我
l_sum :  2.6993712782859802
epoch 120,perplexity 14.870379,time 2.43 sec 
- 分开 你是我 别子我 一九银碗 一九一碗 在一村空 在一蜡空 在一空空 在一蜡空 不不不空 没不不空 没
- 不分开我想你的可样 让我 我想的太笑 快果 在你的太笑 像果在在 你果了我的太笑 你通在在我想多 却这是 
l_sum :  1.3806403875350952
epoch 160,perplexity 3.977448,time 2.55 sec 
- 分开 你已了 你不么难起我 你说啊 是不是你不想活 说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很久
- 不分开我想要你的可样 我要要再想 我不要再想 我不 我不 我不要再想你 爱情走的太快就像龙卷风 不能开受我
